# 2. Исследование надёжности заёмщиков — анализ банковских данных

**Описание проекта:**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Шаг 1. Обзор данных

In [79]:
import pandas as pd
# Прочитаем файл с данными
df = pd.read_csv('/datasets/data.csv')

# Посмотрим структуру данных файла
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [80]:
# Посмотрим структуру файла и объем данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим, что есть пропущенные значения в двух столбцах: *общий трудовой стаж* и *ежемесячный доход*.

In [81]:
# Рассмотрим подробнее состав данных
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Видим следующие аномалии в данных:

1. отрицательное количество детей - ошибка
2. количество детей 20 - необходимо рассмотреть, возможно тоже ошибка
3. отрицательные значения в количестве дней стажа - ошибка
4. нереальное максимальное значение дней стажа (1100 лет) - ошибка
5. среднее значение стажа завышенное (173 года) - рассмотреть причины
6. минимальный возраст 0 лет - пропущены данные

In [82]:
# Проверим столбец Образование на качество заполнения
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Видим, что данные заполнялись *неупорядоченно*, необходимо в дальнейшем привести к единому виду

In [83]:
# Проверим столбец Семейный статус на качество заполнения
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

С данными в столбце *Семейное положение* все в порядке

In [84]:
# Проверим столбец Тип занятости на качество заполнения
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

В столбце *Тип занятости* есть неверные значения (студент, в декрете), но они единичны и не помешают анализу

### Шаг 2.1 Заполнение пропусков

In [85]:
# Проверим, совпадают ли пропущенные значения по строкам
df[(df['total_income'].isnull() == True) & (df['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Предположение подтвердилось - значения пропущены в строках попарно. Возможно, это алгоритм работы программы кредитного эксперта: нет стажа работы, соответственно нет возможности внести доход, но в таком случае агент допустил ошибку, указывая тип занятости не безработный.  
   Количественные переменные (особенно такие, как заработная плата) лучше заполнять медианным значением, так как большие максимальные значения могут завышать среднее значение и искажать реальную картину.

In [86]:
# Подсчитаем долю пропущенных данных
lost_value = (df['children'].count() - df['days_employed'].count())/df['children'].count()
print(f'Доля пропущенных данных: {lost_value:.1%}')

Доля пропущенных данных: 10.1%


In [87]:
# Проверим распределение пропущенных значений по типам занятости
df[(df['total_income'].isnull() == True) & (df['days_employed'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Пропущенные значения неравномерно распределены по группам. При заполнении пропусков необходимо проверить различие медианного дохода по группам и, возможно, заполнять пропуски в зависимости от группы.

In [88]:
# Проверим распределение медианного дохода по типам занятости
df.groupby('income_type')['total_income'].mean()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

Видим, что медианный доход сильно отличается в зависимости от типа занятости, поэтому пропуски необходимо также заполнять по типам занятости.

In [89]:
# Заполним пропущенные значения дохода медианными значениями в зависимости от типа занятости
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))

In [90]:
# Построим сводную таблицу, чтобы причины некорректных значений в столбце Стаж
df_grouped = df.groupby('income_type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})

dict_to_rename = dict(zip(df_grouped.columns.levels[1], ['Количество значений', 'Среднее значение', 'Количество положительных значений']))

df_grouped = df_grouped.rename(columns=dict_to_rename, level=1)
df_grouped

days_employed                   \
                Количество значений Среднее значение   
income_type                                            
безработный                       2    366413.652744   
в декрете                         1     -3296.759962   
госслужащий                    1312     -3399.896902   
компаньон                      4577     -2111.524398   
пенсионер                      3443    365003.491245   
предприниматель                   1      -520.848083   
сотрудник                     10014     -2326.499216   
студент                           1      -578.751554   

                                                   
                Количество положительных значений  
income_type                                        
безработный                                   2.0  
в декрете                                     0.0  
госслужащий                                   0.0  
компаньон                                     0.0  
пенсионер                                  3443.0  
предприниматель                               0.0  
сотрудник                                     0.0  
студент                                       0.0

Из таблицы видно, что положительные значения есть только у безработных и пенсионеров, по остальным значения отрицательные. Таким образом имеем некорректное отображение данных. Для исправления возьмем значения в столбце по модулю.

In [91]:
# Создадим вспомогательный столбец для подсчета соотношения продолжительности стажа к возрасту в зависимости от типа занятости
df['days_employed'] = df['days_employed'].abs()
df['ratio_days_employed'] = df[df['days_employed'].notnull()]['days_employed']/((df['dob_years'])*365.25)

In [92]:
# Заполним пропущенные значения медианным значением из вспомогательного столбца в зависимости от типа занятости
df['days_employed'] = df.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*df['dob_years']*365.25))

In [93]:
# Удаляем вспомогательный столбец и проверяем наличие пропусков
df = df.drop('ratio_days_employed', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим, что все столбцы заполнены, пропусков нет.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [94]:
# Рассмотрим распределение по количеству детей
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим, что отрицательных значений 47, возможно такая же ошибка, как и в стаже, поэтому заменим -1 на 1.
Строк с 20 детьми - 76, так как есть разрыв по количеству между 5 и 20 детьми, скорее всего это ошибка и исправим на 2.

In [95]:
# Заменим количество детей с -1 на 1 и с 20 на 2
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

In [96]:
# Проверим распределение по количеству детей
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Данные по детям выглядят правдоподобно.

In [97]:
# Ранее в таблице было казано, что есть люди с 0 возрастом.Также проверим, сколько значени с возрастом менее 18 лет.
print('Количество строк с возрастом = 0 -', df[df['dob_years'] == 0].count()[0])
print('Количество строк с возрастом < 18 -', df[df['dob_years'] < 18].count()[0])

Количество строк с возрастом = 0 - 101
Количество строк с возрастом < 18 - 101


Количество строк совпадает, значит все "несовершеннолетние" в выборке - строки с пропущенными значениями. Заполним значения медианой в зависимости от типа занятости. Так как возраст имеет целочисленный тип - округлим медиану.

In [98]:
# Добавим пропущенные значения в возрасте округленной медианной взависимости от ипа данных.
df['dob_years'] = df.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))

In [99]:
# Повторно рассмотрим таблицу с данными.
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,2.152500e+04,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,1.523877e+08,43.495238,0.817236,0.972544,0.080883,1.673959e+05
std,0.755528,1.078813e+09,12.230322,0.548138,1.420324,0.272661,9.790695e+04
min,0.000000,0.000000e+00,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1.024652e+03,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2.605748e+03,43.000000,1.000000,0.000000,0.000000,1.519313e+05
75%,1.000000,3.336411e+05,53.000000,1.000000,1.000000,0.000000,2.024175e+05
max,5.000000,9.732179e+09,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод:** Все значения заполнены и выглядят правдоподобно, кроме дней стажа (но в нашем исследовании стаж не важен).

### Шаг 2.3. Изменение типов данных.

In [100]:
# Заменим вещественный тип данных в столбце total_income на целочисленный
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод:** Данные представлены в соответствующих форматах.

### Шаг 2.4. Удаление дубликатов.

In [101]:
# Ранее мы находили, что столбец Образование заполнен неупорядоченно, 
# для дальнейшего исследования необходимо привести его к единому вид. Преобразуем значения в строчные.
df['education'] = df['education'].str.lower()

In [102]:
# Проверим столбец Образование на качество заполнения
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Значения *Образование* приведены к единому виду.

In [103]:
# Подсчитаем количество дубликатов
df.duplicated().sum()

71

In [104]:
# Проверим дубликаты
df[df.duplicated(keep=False)].sort_values(by=['dob_years', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8853,1,1.954434e+07,23,среднее,1,гражданский брак,1,F,сотрудник,0,161380,сыграть свадьбу
15892,0,1.954434e+07,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,161380,сделка с подержанным автомобилем
19321,0,1.954434e+07,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,161380,сделка с подержанным автомобилем
20297,1,1.954434e+07,23,среднее,1,гражданский брак,1,F,сотрудник,0,161380,сыграть свадьбу
3452,0,2.464286e+07,29,высшее,0,женат / замужем,0,M,сотрудник,0,161380,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,8.798957e+09,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,137127,операции со своей недвижимостью
9528,0,8.798957e+09,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,137127,операции со своей недвижимостью
6537,0,9.465544e+09,71,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
7938,0,9.465544e+09,71,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы


Видим явные дубликаты.

In [105]:
# Удалим явные дубликаты
df = df.drop_duplicates()

In [106]:
# Еще раз проверим количество дубликатов
df.duplicated().sum()

0

**Вывод:** Неявные дубликаты возникли из-за неверного занесения вида образования (использование строчных и заглавных букв). Чтобы избегать подобные ошибки, необходимо утвердить формат внесения данных или заносить данные из справочника.
    Явные дубликаты скорее всего - техническая ошибка.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [107]:
# Созадим словарь с видами образования
dict_edu = df[['education_id', 'education']]
dict_edu = dict_edu.drop_duplicates().reset_index(drop=True)
dict_edu

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [108]:
# Созадим словарь с видами семейного положения
dict_family = df[['family_status_id', 'family_status']]
dict_family = dict_family.drop_duplicates().reset_index(drop=True)
dict_family

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [109]:
# Удалим из исходной таблицы столбцы education и family_status
df = df.drop(['education', 'family_status'], axis=1)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод:** Создали словари со значениями вида образования и семейного статуса и удалили описания значений из исходной таблицы. Применение слованрей позволяет уменьшить размер исходной таблицы и упростить операции над ней.

### Шаг 2.6. Категоризация дохода.

In [110]:
# Присвоим всем заемщикам категории в зависимости от их дохода, для этого создадим функцию
def total_income_cat(row):
    if row['total_income'] <= 30000:
        return 'E'
    elif row['total_income'] <= 50000:
        return 'D'
    elif row['total_income'] <= 200000:
        return 'C'
    elif row['total_income'] <= 1000000:
        return 'B'
    else:
        return 'A'

In [111]:
# Применим функцию и запишем данные в новый столбец
df['total_income_category'] = df.apply(total_income_cat, axis=1)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


**Вывод:**
В данном разделе создали функцию и присвоили заемщикам категорию в зависимости от их дохода.

### Шаг 2.7. Категоризация целей кредита.

In [112]:
# Рассмотрим все цели взятия кредита
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [113]:
# Приведем все цели кредитования к 4 категориям, для этого напишем функцию
def purpose_cat(row):
    if 'недв' in row['purpose']:
        return 'операции с недвижимостью'
    if 'жил' in row['purpose']:
        return 'операции с недвижимостью'
    if 'автом' in row['purpose']:
        return 'операции с автомобилем'
    if 'свад' in row['purpose']:
        return 'проведение свадьбы'
    if 'образ' in row['purpose']:
        return 'получение образования'
    else:
        return 'другое'

In [114]:
# Применим функцию и запишем данные в новый столбец
df['purpose_category'] = df.apply(purpose_cat, axis=1)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [115]:
# Проверим, что присвоены все категории
df['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: 
**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [116]:
# Для ответа на вопрос построим сводную таблицу зависимости задолженности по кредиту и количеству детей
df_pivot = df.pivot_table(index='children', values='debt', aggfunc='mean')
print(df_pivot.sort_values(by='debt')) 

              debt
children          
5         0.000000
0         0.075438
3         0.081818
1         0.091658
2         0.094925
4         0.097561


##### Вывод 1:

Заемщики без детей имеют наименьшую задолженность по возврату кредита. У заемщиков с 5 детьми 0 просроченных кредитов, но это связано с малым количеством таких заемщиков.

##### Вопрос 2: 
**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [117]:
# Для ответа на вопрос построим сводную таблицу зависимости задолженности по кредиту и семейному положению
# Так как описание семейного положения вынесено в отдельный справочник, вначале соединим две таблицы
df_merge = df.merge(dict_family, on='family_status_id', how='left')
df_pivot = df_merge.pivot_table(index='family_status', values='debt', aggfunc='mean')
print(df_pivot.sort_values(by='debt')) 

                           debt
family_status                  
вдовец / вдова         0.065693
в разводе              0.071130
женат / замужем        0.075452
гражданский брак       0.093471
Не женат / не замужем  0.097509


##### Вывод 2:

Заемщики холостые и в гражданском браке имеют наибольшую долю просроченных кредитов.

##### Вопрос 3: 
**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [118]:
# Для ответа на вопрос построим сводную таблицу зависимости задолженности по кредиту и уровню дохода
df_pivot = df.pivot_table(index='total_income_category', values='debt', aggfunc='mean')
print(df_pivot.sort_values(by='debt')) 

                           debt
total_income_category          
D                      0.060000
B                      0.069637
A                      0.080000
C                      0.085729
E                      0.090909


##### Вывод 3:

Наименьшую долю просрочек по кредиту имеют заемщики с доходом от 30 тыс. до 50 тыс., а наибольшую долю просрочек - заемщики с самым низким доходом.

##### Вопрос 4: 
**Как разные цели кредита влияют на его возврат в срок?**

In [119]:
# Для ответа на вопрос построим сводную таблицу зависимости задолженности по кредиту и цели займа
df_pivot = df.pivot_table(index='purpose_category', values='debt', aggfunc='mean')
print(df_pivot.sort_values(by='debt')) 

                              debt
purpose_category                  
операции с недвижимостью  0.072334
проведение свадьбы        0.080034
получение образования     0.092200
операции с автомобилем    0.093590


##### Вывод 4:

Операции с недвижимостью дают наименьшую долю задолженности по кредиту.

## Общий вывод:

На вопрос "Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок" можно дать однозначный ответ, что **влияют**.
Наименьшую долю задолженности по кредиту имеют заемщики без детей и вдовые.  